# Credit Card Default Prediction

In this project, the goal is to predict the probability of borrowers defaulting on their credit loans by creating a credit score prediction model. 

## Dataset 
The dataset used in this project contains information on default payments, demographic factors, credit data, history of payment, and bill statements of credit card clients in Taiwan between April 2005 and September 2005.

The dataset was accessed from Kaggle [here](https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset?resource=download).

By the end of this notebook exercise, we hope to have answered the following questions:
1. How does the probability of default payment vary by categories of differnt demographic variables?
2. Which variables are the strongest predictors of default payments?

## Import Important Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

%matplotlib inline

## Load dataset

In [2]:
# read in the dataset
df = pd.read_csv(r"data/raw/UCI_Credit_Card.csv")
# show first five observations
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


There are 25 variables:

- ID: ID of each client
- LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit
- SEX: Gender (1=male, 2=female)
- EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
- MARRIAGE: Marital status (1=married, 2=single, 3=others)
- AGE: Age in years
- PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, … 8=payment delay for eight months, 9=payment delay for nine months and above)
- PAY_2: Repayment status in August, 2005 (scale same as above)
- PAY_3: Repayment status in July, 2005 (scale same as above)
- PAY_4: Repayment status in June, 2005 (scale same as above)
- PAY_5: Repayment status in May, 2005 (scale same as above)
- PAY_6: Repayment status in April, 2005 (scale same as above)
- BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)
- BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)
- BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)
- BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)
- BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)
- BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)
- PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)
- PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)
- PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)
- PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)
- PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)
- PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar)
default.payment.next.month: Default payment (1=yes, 0=no)
Inspiration

In [3]:
# check the shape of the dataset
df.shape

(30000, 25)

In [3]:
# lets look at the type of information for each column
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

In [4]:
df = df.rename(columns={
    "default.payment.next.month":"def_pay"
})

In [5]:
df.memory_usage(deep=True)

Index           132
ID           240000
LIMIT_BAL    240000
SEX          240000
EDUCATION    240000
MARRIAGE     240000
AGE          240000
PAY_0        240000
PAY_2        240000
PAY_3        240000
PAY_4        240000
PAY_5        240000
PAY_6        240000
BILL_AMT1    240000
BILL_AMT2    240000
BILL_AMT3    240000
BILL_AMT4    240000
BILL_AMT5    240000
BILL_AMT6    240000
PAY_AMT1     240000
PAY_AMT2     240000
PAY_AMT3     240000
PAY_AMT4     240000
PAY_AMT5     240000
PAY_AMT6     240000
def_pay      240000
dtype: int64

- we could try optimise the amount of memory used by the dataframe by checking the ranges of the values in each variable (column) to determine if the default data type is appropriate. 

In [6]:
# statistical overview 
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
ID,30000.0,15000.500000,8660.398374,1.0,7500.75,15000.5,22500.25,30000.0
LIMIT_BAL,30000.0,167484.322667,129747.661567,10000.0,50000.00,140000.0,240000.00,1000000.0
SEX,30000.0,1.603733,0.489129,1.0,1.00,2.0,2.00,2.0
EDUCATION,30000.0,1.853133,0.790349,0.0,1.00,2.0,2.00,6.0
MARRIAGE,30000.0,1.551867,0.521970,0.0,1.00,2.0,2.00,3.0
AGE,30000.0,35.485500,9.217904,21.0,28.00,34.0,41.00,79.0
PAY_0,30000.0,-0.016700,1.123802,-2.0,-1.00,0.0,0.00,8.0
PAY_2,30000.0,-0.133767,1.197186,-2.0,-1.00,0.0,0.00,8.0
PAY_3,30000.0,-0.166200,1.196868,-2.0,-1.00,0.0,0.00,8.0
PAY_4,30000.0,-0.220667,1.169139,-2.0,-1.00,0.0,0.00,8.0


In [7]:
# numeric
df["AGE"] = df["AGE"].astype("uint8")
df["SEX"] = df["SEX"].astype("uint8")
df["EDUCATION"] = df["EDUCATION"].astype("uint8")
df["MARRIAGE"] = df["MARRIAGE"].astype("uint8")
df["def_pay"] = df["def_pay"].astype("uint8")

In [8]:
temp_list = [0, 2, 3, 4, 5, 6]
for i in temp_list:
    df[f"PAY_{i}"] = df[f"PAY_{i}"].astype("int8")

In [9]:
temp_list = [1, 2, 3, 4, 5, 6]
for i in temp_list:
    df[f"PAY_AMT{i}"] = df[f"PAY_AMT{i}"].astype("float32")

- managed to reduce the dataframe from 5.9mb to 3mb. Almost a 50% reduction. 

Let us check for:
- missing values in the dataset
- duplicate data

In [10]:
# check for missing values in the dataset
df.isna().sum()

ID           0
LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
def_pay      0
dtype: int64

In [11]:
print(f"Number of duplicate values {df.duplicated().sum()}.")

Number of duplicate values 0.


Correlation Analysis

Important to check how each variable in the dataset is related to each other. Fortunately, all the variables in this dataframe are numeric.

In [12]:
# calculate the correlation matrix 
corr = df.corr()
# extract target - next_month default
next_month_default_corr = corr["def_pay"].sort_values()
next_month_default_corr


LIMIT_BAL   -0.153520
PAY_AMT1    -0.072929
PAY_AMT2    -0.058579
PAY_AMT4    -0.056827
PAY_AMT3    -0.056250
PAY_AMT5    -0.055124
PAY_AMT6    -0.053183
SEX         -0.039961
MARRIAGE    -0.024339
BILL_AMT1   -0.019644
BILL_AMT2   -0.014193
BILL_AMT3   -0.014076
ID          -0.013952
BILL_AMT4   -0.010156
BILL_AMT5   -0.006760
BILL_AMT6   -0.005372
AGE          0.013890
EDUCATION    0.028006
PAY_6        0.186866
PAY_5        0.204149
PAY_4        0.216614
PAY_3        0.235253
PAY_2        0.263551
PAY_0        0.324794
def_pay      1.000000
Name: def_pay, dtype: float64

In [13]:
# plot the heatmap
fig = px.imshow(corr, text_auto=True)
fig.update_layout(
    height=900,
    width=950
)
fig.show()

In [14]:
fig = go.Figure()
fig.add_trace(
    go.Bar(x=next_month_default_corr[:-1].index, y= next_month_default_corr[:-1].values, text=next_month_default_corr[:-1].values)
)
fig.update_layout(
      plot_bgcolor="white",
      height=900,
      width=950,
      margin={
          "l":25,
          "r":25,
          "b":25
      },
      title_text="Price Correlation Distribution <br>",
      title={
          "x":0.5,
          "xanchor":"center",
          "font":{
              "size":14
          }
      },
      xaxis_title="Ride Provider",
      showlegend=False
  )
fig.update_xaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.update_yaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.show()

## EDA

lets look at the distribution of the following:
- target variable
- sex
- marriage
- Age

In [15]:
# target variable
next_month_default = df["def_pay"].value_counts()
fig = go.Figure()
fig.add_trace(
    go.Bar(x=next_month_default.index, y= next_month_default.values, text=next_month_default.values)
)
fig.update_layout(
      plot_bgcolor="white",
      height=450,
      width=475,
      margin={
          "l":25,
          "r":25,
          "b":25
      },
      title_text="Default Next Month Distribution <br>",
      title={
          "x":0.5,
          "xanchor":"center",
          "font":{
              "size":14
          }
      },
      xaxis_title="Target Variable",
      showlegend=False
  )
fig.update_xaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.update_yaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.show()

In [16]:
sex_count = df["SEX"].value_counts()
fig = go.Figure()
fig.add_trace(
    go.Bar(x=sex_count.index, y= sex_count.values, text=sex_count.values)
)
fig.update_layout(
      plot_bgcolor="white",
      height=450,
      width=475,
      margin={
          "l":25,
          "r":25,
          "b":25
      },
      title_text="SEX Distribution <br>",
      title={
          "x":0.5,
          "xanchor":"center",
          "font":{
              "size":14
          }
      },
      xaxis_title="Sex",
      showlegend=False
  )
fig.update_xaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.update_yaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.show()

In [17]:
marriage_count = df["MARRIAGE"].value_counts()
fig = go.Figure()
fig.add_trace(
    go.Bar(x=marriage_count.index, y= marriage_count.values, text=marriage_count.values)
)
fig.update_layout(
      plot_bgcolor="white",
      height=450,
      width=475,
      margin={
          "l":25,
          "r":25,
          "b":25
      },
      title_text="MARRIAGE Distribution <br>",
      title={
          "x":0.5,
          "xanchor":"center",
          "font":{
              "size":14
          }
      },
      xaxis_title="Marriage",
      showlegend=False
  )
fig.update_xaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.update_yaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.show()

In [18]:
age_count = df["AGE"]#.value_counts()
fig = go.Figure()
fig.add_trace(
    go.Histogram(x=age_count.values, text=age_count.values)
)
fig.update_layout(
      plot_bgcolor="white",
      height=450,
      width=475,
      margin={
          "l":25,
          "r":25,
          "b":25
      },
      title_text="AGE Distribution <br>",
      title={
          "x":0.5,
          "xanchor":"center",
          "font":{
              "size":14
          }
      },
      xaxis_title="Age",
      showlegend=False
  )
fig.update_xaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.update_yaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.show()

- from the age histogram plot above, we can see that the customers in the dataset is skewed towards age range 20-45. The next stpe would to see how the target variable affects the visualisations created above. 

In [19]:
sex_target = (df.
              groupby(["SEX","def_pay"])["ID"]
              .count()
              .unstack())


In [20]:
sex_target[0]

SEX
1     9015
2    14349
Name: 0, dtype: int64

In [21]:

fig = go.Figure()
fig.add_trace(
    go.Bar(x=sex_target.index, y=sex_target[0], text=sex_target[0])
)

fig.add_trace(
    go.Bar(x=sex_target.index, y=sex_target[1], text=sex_target[1])
)
fig.update_layout(
      plot_bgcolor="white",
      height=450,
      width=475,
      margin={
          "l":25,
          "r":25,
          "b":25
      },
      title_text="Sex Distribution by Default Status <br>",
      title={
          "x":0.5,
          "xanchor":"center",
          "font":{
              "size":14
          }
      },
      xaxis_title="Sex",
      showlegend=False
  )
fig.update_xaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.update_yaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.show()

In [22]:
marriage_target = (df.
              groupby(["MARRIAGE","def_pay"])["ID"]
              .count()
              .unstack())

In [23]:

fig = go.Figure()
fig.add_trace(
    go.Bar(x=marriage_target.index, y=marriage_target[0], text=marriage_target[0])
)

fig.add_trace(
    go.Bar(x=marriage_target.index, y=marriage_target[1], text=marriage_target[1])
)
fig.update_layout(
      barmode="stack",
      plot_bgcolor="white",
      height=450,
      width=475,
      margin={
          "l":25,
          "r":25,
          "b":25
      },
      title_text="Marriage Distribution by Default Status <br>",
      title={
          "x":0.5,
          "xanchor":"center",
          "font":{
              "size":14
          }
      },
      xaxis_title="Marriage",
      showlegend=False
  )
fig.update_xaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.update_yaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.show()

In [38]:
# create bins for the age
age_bins = [20,30,40,50,60,70,80]
# create string for each bin
age_bins_str = ["21-30","31-40","41-50","51-60","61-70","71-80"]



# bin the age values into discrete intervals
df["age_bins"] = pd.cut(x= df["AGE"], bins = age_bins, labels=age_bins_str, right=True)

In [40]:

age_target = (df.
              groupby(["age_bins","def_pay"])["ID"]
              .count()
              .unstack())

C:\Users\user\AppData\Local\Temp\ipykernel_6000\3683945478.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [41]:
age_target

def_pay,0,1
age_bins,,
21-30,8542,2471
31-40,8524,2189
41-50,4606,1399
51-60,1493,504
61-70,189,68
71-80,10,5


In [44]:

fig = go.Figure()
fig.add_trace(
    go.Bar(x=age_target.index, y=age_target[0], text=age_target[0], name="non-defaulters")
)

fig.add_trace(
    go.Bar(x=age_target.index, y=age_target[1], text=age_target[1],name="defaulters")
)

fig.update_layout(
      barmode="stack",
      bargap=0.1,
      plot_bgcolor="white",
      height=450,
      width=475,
      margin={
          "l":25,
          "r":25,
          "b":25
      },
      title_text="Age Distribution by Default Status <br>",
      title={
          "x":0.5,
          "xanchor":"center",
          "font":{
              "size":14
          }
      },
      xaxis_title="Age",
      showlegend=True)
fig.update_xaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.update_yaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.show()

From the plot above, we can see that for this customer base, the likelihood of defaulting is negatively correlated with age, i.e., as the customer base age increases, the less number of defaulters reduces. This means that the younger the customer the more likely they are to default on their loan. 

- let us plot how the score of repayment status history influences the target variable. 

In [56]:
temp_list = [0, 2, 3, 4, 5, 6]
df_list = []
for month in temp_list:
    temp_df = (df.
     groupby([f"PAY_{month}","def_pay"])["ID"]
              .count()
              .unstack()
              .rename(columns={0:"non_defaulters",
                               1:"defaulters"})
      )
    temp_df = (temp_df
               .assign(
                   defaulters = (temp_df["defaulters"]/(temp_df["defaulters"]+temp_df["non_defaulters"]))*100,
                   non_defaulters = (temp_df["non_defaulters"]/(temp_df["defaulters"]+temp_df["non_defaulters"]))*100
               ))
    df_list.append(temp_df)

In [29]:
c = -1
months = np.array(["September","August","July","June","May","April"])
for x in [1,2]:
    for k in [1,2,3]:
        c+=1
        print(x,k,c)

1 1 0
1 2 1
1 3 2
2 1 3
2 2 4
2 3 5


In [57]:

df_list[0]

def_pay,non_defaulters,defaulters
PAY_0,,
-2,86.770569,13.229431
-1,83.221949,16.778051
0,87.188709,12.811291
1,66.052061,33.947939
2,30.858643,69.141357
3,24.223602,75.776398
4,31.578947,68.421053
5,50.000000,50.000000
6,45.454545,54.545455


In [53]:
df_list[0]["defaulters"] = (df_list[0]["defaulters"]/ df_list[0]["total"])*100
df_list[0]

def_pay,non-defaulters,defaulters,total
PAY_0,,,
-2,2394,13.229431,2759
-1,4732,16.778051,5686
0,12849,12.811291,14737
1,2436,33.947939,3688
2,823,69.141357,2667
3,78,75.776398,322
4,24,68.421053,76
5,13,50.000000,26
6,5,54.545455,11


In [59]:
fig = make_subplots(2,3,
                   subplot_titles =(
                    "Repayment Status - September",
                    "Repayment Status - August",
                    "Repayment Status - July",
                    "Repayment Status - June",
                    "Repayment Status - May",
                    "Repayment Status - April",
                   )
)
months = np.array(["September","August","July","June","May","April"])
c=-1
for row_ in [1,2]:
    for col_ in [1,2,3]:
        c+=1
        fig.add_trace(
            go.Bar(name=f"Repayment Status- {months[c]} - no default", x=df_list[c].index, y=df_list[c]["non_defaulters"].values, marker_color="blue"), row=row_, col=col_
        )
        fig.add_trace(
            go.Bar(name=f"Repayment Status - {months[c]} - default", x=df_list[c].index, y=df_list[c]["defaulters"].values, marker_color="red"), row=row_, col=col_
        )
fig.update_layout(
      barmode='relative',
      plot_bgcolor="white",
      height=900,
      width=950,
      margin={
          "l":25,
          "r":25,
          "b":25
      },
      title_text="Repayment Status from Between April and September 2005 with a scale in delay of payment. <br>",
      title={
          "x":0.5,
          "xanchor":"center",
          "font":{
              "size":14
          }
      },
      # xaxis_title="Ride Provider",
      showlegend=False
  )
fig.update_xaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.update_yaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.show()

Lets look at the change in default status by age and sex.

In [70]:
# px express
fig = px.histogram(df, x="AGE",facet_row="SEX", facet_col="def_pay", nbins=10)
fig.show()

From the plot above we can infer, that in all age bin, the women of this bank tend to have higher occurrence of defaulting. 